In [1]:
import pandas as pd
import datetime
import os
import numpy as np
import hashlib
import gc
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Bing_LiveRamp_Test'

In [5]:
thismonday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday())
thismonday
last_week_end_saturday=thismonday-datetime.timedelta(days=2)

In [6]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            


In [7]:
posibble_recent_folder="/home/jian/BigLots/MediaStorm_"+str(last_week_end_saturday)+"/"
daily_files_recent=[x for x in list(recursive_file_gen(posibble_recent_folder)) if "Daily" in x]

list_1_after_201806_2019=[x for x in list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/")) if ("aily" in x) & (".txt" in x) ]
list_1_after_201806_2019=[x for x in list_1_after_201806_2019 if str(last_week_end_saturday) in x]


daily_files_last_week=daily_files_recent+list_1_after_201806_2019
if len(daily_files_last_week)==1:
    daily_file_last_week=daily_files_last_week[0]
    print("Good to load")
else:
    daily_file_last_week=np.nan
    print("Last week daily data not avaiable")

Good to load


In [8]:
sales_daily_lastweek=pd.read_table(daily_file_last_week,sep="|",dtype=str,usecols=['location_id','customer_id_hashed','transaction_dt','subclass_transaction_amt'])
sales_daily_lastweek=sales_daily_lastweek[~pd.isnull(sales_daily_lastweek['customer_id_hashed'])]
sales_daily_lastweek=sales_daily_lastweek[sales_daily_lastweek['location_id']!="6990"]
sales_daily_lastweek['subclass_transaction_amt']=sales_daily_lastweek['subclass_transaction_amt'].astype(float)
sales_daily_lastweek_agg=sales_daily_lastweek.groupby(['customer_id_hashed','transaction_dt'])['subclass_transaction_amt'].sum().to_frame().reset_index()
sales_daily_lastweek_agg=sales_daily_lastweek_agg.rename(columns={"transaction_dt":"Timestamp","subclass_transaction_amt":"Conversion_Amount"})
sales_daily_lastweek_agg['Timestamp']=sales_daily_lastweek_agg['Timestamp'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())


In [9]:
Master_2019_weekly=[x for x in list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/")) if ("aster" in x) & (".txt" in x) ]
Master_2018_weekly=[x for x in list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/")) if ("aster" in x) & (".txt" in x) ]

weekly_df=pd.DataFrame({"file_path":Master_2019_weekly+Master_2018_weekly})
weekly_df['date']=weekly_df['file_path'].apply(lambda x: x.split("_by_weeks/MediaStorm_")[1][:10])
weekly_df['date']=weekly_df['date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
weekly_df=weekly_df.sort_values("date",ascending=False).reset_index()
del weekly_df['index']
weekly_df

,file_path,date
0,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-01-26
1,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-01-12
2,/home/jian/BigLots/2018_by_weeks/MediaStorm_20...,2018-12-29
3,/home/jian/BigLots/2018_by_weeks/MediaStorm_20...,2018-12-15
4,/home/jian/BigLots/2018_by_weeks/MediaStorm_20...,2018-12-01
5,/home/jian/BigLots/2018_by_weeks/MediaStorm_20...,2018-11-17
6,/home/jian/BigLots/2018_by_weeks/MediaStorm_20...,2018-11-03
7,/home/jian/BigLots/2018_by_weeks/MediaStorm_20...,2018-10-20
8,/home/jian/BigLots/2018_by_weeks/MediaStorm_20...,2018-10-06
9,/home/jian/BigLots/2018_by_weeks/MediaStorm_20...,2018-09-22


In [10]:
data_0=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStorm_Lapsed_Reward_Member_Master_from2014-08-26to2017-02-26.zip",
                     dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','customer_zip_code']).drop_duplicates()
print(data_0.shape)


(1873409, 3)


In [11]:
data_1=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStormCustTot-hashed-email.txt",
                     dtype=str,header=None,usecols=[0,1,5])
data_1.columns=['customer_id_hashed','email_address_hash','customer_zip_code']
data_1['customer_id_hashed']=data_1['customer_id_hashed'].apply(lambda x: hashlib.sha256(x.encode('utf-8')).hexdigest())
print(data_1.shape)

(19337597, 3)


In [12]:
data_2 = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/'+'MediaStormCustomerTransactionTotals_2018-01-09_2018-03-31.txt',
                     sep = ',',dtype = str,usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
print(data_2.shape)
data_3 = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/'+'Existing Reward Member Master as of 2018-06-05.txt',
                     dtype = str,sep = '|',usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
print(data_3.shape)
data_4 = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/'+'New Reward Member Master as of 2018-06-05.txt',
                     dtype = str,sep = '|',usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
print(data_4.shape)

(7255226, 3)
(5759051, 3)
(659127, 3)


In [13]:
master_old=data_4.append(data_3).append(data_2).append(data_1).append(data_0).drop_duplicates()
del data_4
del data_3
del data_2
del data_1
del data_0
gc.collect()

42

In [14]:
all_weekly_biweekly_master_file=pd.DataFrame()
for file_path in weekly_df['file_path'].tolist():
    df=pd.read_table(file_path,dtype=str,usecols=['customer_id_hashed','email_address_hash','customer_zip_code'],sep="|")
    all_weekly_biweekly_master_file=all_weekly_biweekly_master_file.append(df)

all_weekly_biweekly_master_file=all_weekly_biweekly_master_file.append(master_old)
print(all_weekly_biweekly_master_file.shape)
print(len(all_weekly_biweekly_master_file['customer_id_hashed'].unique()))

all_weekly_biweekly_master_file=all_weekly_biweekly_master_file.drop_duplicates('customer_id_hashed')
print(all_weekly_biweekly_master_file.shape)

(24277095, 3)
24242338
(24242338, 3)


In [15]:
all_weekly_biweekly_master_file['customer_id_hashed'].apply(lambda x: len(x)).unique()

array([64])

In [16]:
all_weekly_biweekly_master_file['email_address_hash'].apply(lambda x: len(x)).unique()

array([64])

In [17]:
print(all_weekly_biweekly_master_file.shape)

(24242338, 3)


In [18]:
print(sales_daily_lastweek_agg.shape)

sales_daily_lastweek_agg=pd.merge(sales_daily_lastweek_agg,all_weekly_biweekly_master_file,on="customer_id_hashed",how="left").rename(columns={"email_address_hash":"Email_1","customer_zip_code":"Zip"})
sales_daily_lastweek_agg.head(2)


print("Null Email rows excluded: "+str(sales_daily_lastweek_agg[pd.isnull(sales_daily_lastweek_agg['Email_1'])].shape[0]))
sales_daily_lastweek_agg=sales_daily_lastweek_agg[~pd.isnull(sales_daily_lastweek_agg['Email_1'])]
del sales_daily_lastweek_agg['customer_id_hashed']

(1274895, 3)
Null Email rows excluded: 72140


In [19]:
sales_daily_lastweek_agg=sales_daily_lastweek_agg[["Email_1","Zip","Timestamp", "Conversion_Amount"]]
sales_daily_lastweek_agg['Conversion_Amount']=sales_daily_lastweek_agg['Conversion_Amount'].apply(lambda x: np.round(x,2)).astype(str)
sales_daily_lastweek_agg['Conversion_Amount']=sales_daily_lastweek_agg['Conversion_Amount'].apply(lambda x: x.split(".")[0]+"."+x.split(".")[1].ljust(2,"0"))
sales_daily_lastweek_agg['Product_Group']="In_Store"



In [20]:
sales_daily_lastweek_agg.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Bing_LiveRamp_Test/BL_LR_Bing_Store_Sales_JL_"+str(datetime.datetime.now().date())+".txt",index=False,sep="|")


In [21]:
sorted(sales_daily_lastweek_agg.columns.tolist())==sorted(["Zip", "Email_1", "Timestamp", "Conversion_Amount","Product_Group"])

True

In [22]:
x=sales_daily_lastweek_agg[sales_daily_lastweek_agg['Conversion_Amount'].astype(str).apply(lambda x: len(x.split(".")[1]))!=2]
x.head(2)

,Email_1,Zip,Timestamp,Conversion_Amount,Product_Group


In [23]:
sales_daily_lastweek_agg.shape

(1202755, 5)

In [24]:
sales_daily_lastweek_agg.columns.tolist()

['Email_1', 'Zip', 'Timestamp', 'Conversion_Amount', 'Product_Group']

In [25]:
sales_daily_lastweek_agg.head(2)

,Email_1,Zip,Timestamp,Conversion_Amount,Product_Group
0,e7b239d884e48cdd957205e0ffb8e793bb7b8a391dc102...,21229,2019-01-26,7.92,In_Store
1,7518e767bba85d75e674b111c6edfba4c99e55667dac57...,06455,2019-01-20,13.08,In_Store


In [26]:
sales_daily_lastweek_agg['Timestamp'].unique()

array([datetime.date(2019, 1, 26), datetime.date(2019, 1, 20),
       datetime.date(2019, 1, 21), datetime.date(2019, 1, 25),
       datetime.date(2019, 1, 22), datetime.date(2019, 1, 23),
       datetime.date(2019, 1, 24)], dtype=object)

In [27]:
sales_daily_lastweek_agg.shape

(1202755, 5)

In [28]:
len(sales_daily_lastweek_agg['Email_1'].unique())

1093804